In [22]:
import numpy as np
import matplotlib.pyplot as plt



# Define simulation function
def simulate_boarding(assign_type):
    # Initialize Constants
    # Define number of rows and columns
    n_rows = 33
    n_cols = 6
    
    # Calculate number of passengers
    n_pass = n_rows * n_cols

    # Create seat matrix, using -1 to represent unoccupied seats
    seats = np.zeros((n_rows, n_cols))
    seats[:, :] = -1

    # Create aisle array
    aisle_q = np.zeros(n_rows)
    aisle_q[:] = -1

    # Create initial passenger number queue
    pass_q = [int(i) for i in range(n_pass)]
    pass_q = np.array(pass_q)

    # Define seating disobedience and queue disobedience constants
    seat_disobediance = 0.05  # Smaller number for seating disobedience
    queue_disobedience = 0.1  # Adjust the value as needed

    # Calculate the extent of shuffling based on queue_disobedience
    shuffle_factor = int(queue_disobedience * n_pass)
    shuffle_factor = min(shuffle_factor, n_pass)

    # Shuffle the first 'shuffle_factor' passengers in the queue
    if shuffle_factor > 0:
        np.random.shuffle(pass_q[:shuffle_factor])
    
    # Account for passenger ages and physical passenger disabilities
    # Define the parameters for the age distribution
    mean_age = 40
    stddev_age = 15

    # Generate passenger ages based on the custom distribution
    passenger_ages = np.random.normal(mean_age, stddev_age, n_pass)
    # Ensure the ages are non-negative
    passenger_ages = np.maximum(passenger_ages, 0)
    # Round the ages to whole numbers
    passenger_ages = np.round(passenger_ages)
    
    passenger_disabilities = np.random.choice([0, 1], n_pass, p=[0.95, 0.05])  # 10% chance of having a disability

    # Define group sizes based on your criteria
    group_sizes = np.random.choice([1, 2, 3, 4], n_pass, p=[0.3, 0.3, 0.2, 0.2])  # 4 least common, 3 more common, 2 more common, and 1 most common

    # Create array for seat nos
    row_q_init = np.zeros(n_pass)
    col_q_init = np.zeros(n_pass)

    # Let's create moveto arrays
    moveto_loc = np.zeros(n_pass)
    moveto_time = np.zeros(n_pass)

    moveto_loc_dict = {i: j for i in pass_q for j in moveto_loc}
    moveto_time_dict = {i: j for i in pass_q for j in moveto_time}


    def AssignSeats(rq, cq, assign_type, group_sizes, n_pass=n_pass, n_rows=n_rows):
        group_start = 0
        for group_size in set(group_sizes):
            group_indices = np.where(group_sizes == group_size)[0]
            group_indices = group_indices[group_start:group_start + len(group_indices) // group_size * group_size]
            group_start += len(group_indices)
            if assign_type == "Random":
                available_seats = [(row, col) for row in range(n_rows) for col in range(n_cols) if seats[row, col] == -1]
                group_indices = np.random.permutation(group_indices)
                for i, index in enumerate(group_indices):
                    row, col = available_seats[i]
                    rq[index] = row
                    cq[index] = col
                    seats[row, col] = index

            elif assign_type == "BTF":
                # Implement Back-to-Front method here
                num_groups = n_rows // group_size
                for i in range(num_groups):
                    group = group_indices[i * group_size:(i + 1) * group_size]
                    group.reverse()
                    for j, index in enumerate(group):
                        row = i * group_size + j
                        cq[index] = col = i % 2 * (n_cols - 1)
                        rq[index] = row
                        seats[row, col] = index

            elif assign_type == "FTB":
                # Implement Front-to-Back method here
                num_groups = n_rows // group_size
                for i in range(num_groups):
                    group = group_indices[i * group_size:(i + 1) * group_size]
                    group.reverse()
                    for j, index in enumerate(group):
                        row = (i + 1) * group_size - 1 - j
                        cq[index] = col = i % 2 * (n_cols - 1)
                        rq[index] = row
                        seats[row, col] = index

            elif assign_type == "WMA":
                # Implement Window-Middle-Aisle method here
                wma_seats = [(row, col) for row in range(n_rows) for col in [0, 5, 1, 4, 2, 3]]
                group_indices = np.random.permutation(group_indices)
                for i, index in enumerate(group_indices):
                    row, col = wma_seats[i]
                    rq[index] = row
                    cq[index] = col
                    seats[row, col] = index

            elif assign_type == "Southwest":
                # Implement Southwest method here
                num_groups = n_rows // group_size
                for i in range(num_groups):
                    group = group_indices[i * group_size:(i + 1) * group_size]
                    window_seats = [(i, 0) for i in range(i * group_size, (i + 1) * group_size)]
                    aisle_seats = [(i, 2) for i in range(i * group_size, (i + 1) * group_size)]
                    group_seats = window_seats + aisle_seats
                    np.random.shuffle(group_seats)
                    for j, index in enumerate(group):
                        row, col = group_seats[j]
                        rq[index] = row
                        cq[index] = col
                        seats[row, col] = index


        return rq, cq


    ####################################################

    #Create function to move passengers into the aircraft
    def MoveToAisle(t, aisle_q, pass_q, sum_time):
        if t > sum_time[0]:
            if aisle_q[0] == -1:
                aisle_q[0] = pass_q[0].copy()
                pass_q = np.delete(pass_q, 0)
                sum_time = np.delete(sum_time, 0)
        return aisle_q, pass_q, sum_time


    ###################################################
    
    def get_walk_speed(age, disability):
        if age > 70 or age < 8:
            if disability == 1:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and disability
            else:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and no disability
        elif disability == 1:
            mean_time = 1.5  # 50% increase for passengers with disabilities
        else:
            mean_time = 1.0  # Default mean_time for other passengers

        stddev_time = 0.2
        walk_speed = np.random.normal(loc=mean_time, scale=stddev_time)
        return walk_speed
    
    # Create an array of walking speeds based on age and disability
    walk_speeds = [get_walk_speed(age, disability) for age, disability in zip(passenger_ages, passenger_disabilities)]


    ##################################################

    # Assign seating order
    group_sizes = np.random.choice([1, 2, 3, 4], n_pass, p=[0.6, 0.2, 0.1, 0.1])
    row_q, col_q = AssignSeats(row_q_init, col_q_init, assign_type, group_sizes)

    # Create array for times
    mean_time = 1.
    stddev_time = 0.2
    # Random numbers from a normal distribution with mean = mean_time and standard deviation stddev_time
    time_q = walk_speeds

    # Define multipliers (+2 for stowing luggage)
    # Empty row
    empty_mult = 1 + 2
    # Aisle occupied
    aisle_mult = 4 + 2
    # Middle occupied
    middle_mult = 5 + 2
    # Aisle and middle occupied
    aisle_middle_mult = 7 + 2

    # Create seat and speed dictionary
    pass_dict = {}
    time_dict = {}

    seat_nos = np.column_stack((row_q, col_q))
    for i in range(n_pass):
        pass_dict[i] = seat_nos[i]

    for i in range(n_pass):
        time_dict[i] = time_q[i]

    # Create sum time array
    sum_time = np.zeros(n_pass)
    for i in range(n_pass):
        sum_time[i] = sum(time_q[:i+1])

    # Define initial conditions
    time = 0
    time_step = 0.1
    exit_sum = np.sum(pass_q)
    pass_sum = np.sum(seats)

    while pass_sum != exit_sum:
        if pass_q.size != 0:
            aisle_q, pass_q, sum_time = MoveToAisle(time, aisle_q, pass_q, sum_time)

        for passg in aisle_q:
            if passg != -1:
                row = int(np.where(aisle_q == passg)[0][0])
                if moveto_time_dict[passg] != 0:
                    if time > moveto_time_dict[passg]:
                        if moveto_loc_dict[passg] == "a":
                            if aisle_q[row + 1] == -1:
                                aisle_q[row + 1] = passg
                                aisle_q[row] = -1
                                moveto_loc_dict[passg] = 0
                                moveto_time_dict[passg] = 0
                        elif moveto_loc_dict[passg] == "s":
                            passg_row = int(pass_dict[passg][0])
                            passg_col = int(pass_dict[passg][1])
                            # Generate a random number between 0 and 1
                            rand_num = np.random.rand()
                            # Check if the random number is less than or equal to seat_disobediance
                            if rand_num <= seat_disobediance:
                                # Assign the passenger to a random seat that is not their own
                                # Assume there are 6 columns in each row
                                wrong_col = np.random.randint(0, 6)
                                # Make sure the wrong column is not the same as the correct column
                                while wrong_col == passg_col:
                                    wrong_col = np.random.randint(0, 6)
                                # Assign the passenger to the wrong seat
                                seats[passg_row, wrong_col] = passg
                            else:
                                # Assign the passenger to their correct seat
                                seats[passg_row, passg_col] = passg
                            aisle_q[row] = -1
                elif moveto_time_dict[passg] == 0:
                    passg_row = int(pass_dict[passg][0])
                    passg_col = int(pass_dict[passg][1])
                    if passg_row == row:
                        moveto_loc_dict[passg] = "s"
                        if passg_col == 0:
                            if seats[passg_row, 1] != -1 and seats[passg_row, 2] != -1:
                                moveto_time_dict[passg] = time + aisle_middle_mult * time_dict[passg]
                            elif seats[passg_row, 1] != -1:
                                moveto_time_dict[passg] = time + middle_mult * time_dict[passg]
                            elif seats[passg_row, 2] != -1:
                                moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                            else:
                                moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                        elif passg_col == 5:
                            if seats[passg_row, 4] != -1 and seats[passg_row, 3] != -1:
                                moveto_time_dict[passg] = time + aisle_middle_mult * time_dict[passg]
                            elif seats[passg_row, 4] != -1:
                                moveto_time_dict[passg] = time + middle_mult * time_dict[passg]
                            elif seats[passg_row, 3] != -1:
                                moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                            else:
                                moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                        elif passg_col == 1:
                            if seats[passg_row, 2] != -1:
                                moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                            else:
                                moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                        elif passg_col == 4:
                            if seats[passg_row, 3] != -1:
                                moveto_time_dict[passg] = time + aisle_mult * time_dict[passg]
                            else:
                                moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                        elif passg_col == 2 or passg_col == 3:
                            moveto_time_dict[passg] = time + empty_mult * time_dict[passg]
                    elif passg_row != row:
                        moveto_loc_dict[passg] = "a"
                        moveto_time_dict[passg] = time + time_dict[passg]

        # Iteration timekeeping
        time += time_step
        pass_sum = np.sum(seats)


        """
        print("Current state:")
        print("Time:", time)
        print("Passenger queue:", pass_q)
        print("Aisle queue:", aisle_q)
        """
        

    return time


In [ ]:
# Number of simulations
num_simulations = 10

boarding_times = []

for _ in range(num_simulations):
    result = simulate_boarding("Random")
    if result > 0:
        boarding_times.append(result)
    else:
        print("negative number!")

# Calculate the average time
average_time = sum(boarding_times) / len(boarding_times)

# Calculate percentiles
percentile_5 = np.percentile(boarding_times, 5)
percentile_95 = np.percentile(boarding_times, 95)

print("Average boarding time for {} simulations: {:.2f}".format(len(boarding_times), average_time))
print("5th percentile boarding time: {:.2f}".format(percentile_5))
print("95th percentile boarding time: {:.2f}".format(percentile_95))

# Create a histogram of boarding times
plt.hist(boarding_times, bins=20, color='blue', alpha=0.7)
plt.xlabel("Boarding Time")
plt.ylabel("Frequency")
plt.title("Histogram of Boarding Times")
plt.grid(True)
plt.show()